In [33]:
# load dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [34]:
# Load data and select trainging hour and test hour
trainhour = 5
testhour = 7
df = pd.read_csv('combined.csv')
cols = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
cols10 = ['31', '32', '33', '34', '35', '36', '37', '38', '39', '40']
df['begin'] = df[cols].sum(axis=1)
df['sum10'] = df[cols10].sum(axis=1)
data = df.drop(df[(df['hour'] != trainhour)].index)
testdata = df.drop(df[(df['hour'] != testhour)].index)
del df

In [35]:
# Categorize the 'sum' column the number of PIPs over selected range
def setlabels(x):
    if x < -10:
        return -1
    elif x > 10:
        return 1
    else:
        return 0

data["labels"] = data["sum10"].apply(setlabels)
data.head()

,Unnamed: 0,year,month,day,hour,minute,Weekday,Change,1,2,...,57,58,59,60,dow_open,euf_open,us_6mn,begin,sum10,labels
298,298,2018,1,2,5,0,3,3.2,3.2,5.0,...,1.9,-0.8,-1.4,-0.7,2.480935,2.343,1.61,0.6,6.6,0
299,299,2018,1,2,5,1,3,5.0,5.0,1.0,...,-0.8,-1.4,-0.7,0.1,2.480935,2.343,1.61,-2.2,5.2,0
300,300,2018,1,2,5,2,3,1.0,1.0,-4.6,...,-1.4,-0.7,0.1,1.5,2.480935,2.343,1.61,-6.4,0.1,0
301,301,2018,1,2,5,3,3,-4.6,-4.6,-2.0,...,-0.7,0.1,1.5,-0.9,2.480935,2.343,1.61,-8.5,-2.0,0
302,302,2018,1,2,5,4,3,-2.0,-2.0,-1.9,...,0.1,1.5,-0.9,-1.8,2.480935,2.343,1.61,-3.8,2.5,0


In [36]:
X_data = data.iloc[:,18:38]
X_data['begin'] = data['begin']
X_data['dow_open'] = data['dow_open']
X_data['euf_open'] = data['euf_open']
X_data['us_6mn'] = data['us_6mn']
X_data.head()

,11,12,13,14,15,16,17,18,19,20,...,25,26,27,28,29,30,begin,dow_open,euf_open,us_6mn
298,0.4,0.8,-1.1,0.1,0.2,-0.5,-1.1,-2.0,0.0,-1.7,...,1.2,1.6,1.5,1.3,0.1,1.7,0.6,2.480935,2.343,1.61
299,0.8,-1.1,0.1,0.2,-0.5,-1.1,-2.0,0.0,-1.7,1.5,...,1.6,1.5,1.3,0.1,1.7,-0.4,-2.2,2.480935,2.343,1.61
300,-1.1,0.1,0.2,-0.5,-1.1,-2.0,0.0,-1.7,1.5,-0.7,...,1.5,1.3,0.1,1.7,-0.4,2.7,-6.4,2.480935,2.343,1.61
301,0.1,0.2,-0.5,-1.1,-2.0,0.0,-1.7,1.5,-0.7,-1.9,...,1.3,0.1,1.7,-0.4,2.7,2.2,-8.5,2.480935,2.343,1.61
302,0.2,-0.5,-1.1,-2.0,0.0,-1.7,1.5,-0.7,-1.9,-0.2,...,0.1,1.7,-0.4,2.7,2.2,-1.0,-3.8,2.480935,2.343,1.61


In [37]:
# Get X, y for model
X = X_data.to_numpy()

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
y = data['labels']
label_encoder = LabelEncoder()
label_encoder.fit(y)
y_encoded = label_encoder.transform(y)
y = to_categorical(y_encoded)

In [38]:
# Split data into training and test
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
#X_train[0:1]

In [39]:
# Create model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=X.shape[1]))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=y.shape[1], activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               2500      
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 303       
Total params: 12,903
Trainable params: 12,903
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Fit model
model.fit(X, y, epochs=5, shuffle=True, verbose=0)

In [41]:
# Model loss and Accuray
model_loss, model_accuracy = model.evaluate(X, y, verbose=0)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.15805517490011928, Accuracy: 0.9624145030975342


In [42]:
formodel = testdata.iloc[:,18:38]
formodel['begin'] = testdata['begin']
formodel['dow_open'] = testdata['dow_open']
formodel['euf_open'] = testdata['euf_open']
formodel['us_6mn'] = testdata['us_6mn']
formodel = formodel.to_numpy()
formodel.shape

(15060, 24)

In [43]:
# Pull out validation data and make predicts
testdata['class'] = label_encoder.inverse_transform(model.predict_classes(formodel))
testdata["sum10"].describe()

count    15060.000000
mean         0.139502
std          6.265401
min        -95.000000
25%         -3.100000
50%          0.200000
75%          3.300000
max         54.400000
Name: sum10, dtype: float64

In [44]:
testdata = testdata.drop(testdata[(testdata['class'] == 0)].index)
testdata["sum10"].describe()

count    62.000000
mean     -1.990323
std      16.295989
min     -95.000000
25%      -8.050000
50%       0.050000
75%       6.350000
max      20.900000
Name: sum10, dtype: float64

In [45]:
totalrows = testdata.shape[0]
totalcols = testdata.shape[1]
profit = []
for i in range(totalrows):
    temp = 0
    for j in range(10):
        temp += testdata.iloc[i,38+j]
        if (testdata.iloc[i,totalcols-1] == -1):
            if(temp < 2):
              break  
        else:
            if(temp < -2):
              break
        
    if (testdata.iloc[i,totalcols-1] == -1):
        temp = -temp
    profit.append(temp)

print("Est Profit: " + str(sum(profit)))

Est Profit: -25.30000000000001


In [46]:
totalrows = testdata.shape[0]
profit = []
for i in range(totalrows):
    temp = testdata.iloc[i,totalcols-1] * testdata.iloc[i,totalcols-2]
    profit.append(temp)

print("Est Profit: " + str(sum(profit)))

Est Profit: -41.19999999999999


In [47]:
j = 1
print(str(testdata.iloc[j,totalcols-1]) + " " + str(testdata.iloc[j,totalcols-2]))

1 7.6000000000000005


In [48]:
#testdata.to_csv('updated.csv')